In [4]:
import requests
import pandas as pd
import os
from pathlib import Path
import numpy as np

In [5]:
baseurl = "https://api.wikiwho.net/en/api/v1.0.0-beta/"
content = "Mama's_Family"
def _url(url, path):
    return url + path
revisions_url = _url( baseurl,"rev_ids/")
mama_revisions_url = _url(revisions_url, content+"/")
params = {"editor": "true", "timestamp": "true"}
response = requests.get(mama_revisions_url, params= params)

In [6]:
response.url

"https://api.wikiwho.net/en/api/v1.0.0-beta/rev_ids/Mama's_Family/?editor=true&timestamp=true"

In [7]:
revisons = pd.DataFrame(response.json()["revisions"])

In [8]:
revisons.head()

,editor,id,timestamp
0,0|172.194.5.181,3317807,2004-04-24T07:40:27Z
1,0|172.194.5.181,3317866,2004-04-24T07:49:11Z
2,41723,3317921,2004-04-24T07:58:03Z
3,44476,3317993,2004-04-24T08:09:12Z
4,41723,3493274,2004-04-24T08:21:07Z


In [9]:
revisons["id"].describe()

count    2.499000e+03
mean     3.352492e+08
std      2.328586e+08
min      3.317807e+06
25%      1.352675e+08
50%      2.151685e+08
75%      5.464091e+08
max      8.182558e+08
Name: id, dtype: float64

In [10]:
rev_content_url = _url(baseurl,"rev_content/")
mama_rev_content_url = _url(rev_content_url, content+"/")
filename = content + ".h5"

#### Main cell to calculate the deleted and edited positions

In [22]:
import time
params ={ "o_rev_id": "true", "editor": "false", "token_id": "true","in": "true", "out": "true" }
revisions = list(revisons.head(50)["id"])
with pd.HDFStore(filename, 'a') as store:
    for itr in range(len(revisions)):
        revision = str(revisions[itr])
        url = _url(mama_rev_content_url, revision+"/")
        response = requests.get(url, params= params)
        time.sleep(3)
        if response.status_code == requests.codes.ok :
            df = pd.DataFrame.from_dict(response.json()["revisions"][0][revision]["tokens"])
            df["inserted"] = df["in"].apply(lambda x: int(revision) in  set(x))
            df["inserted"] = np.logical_or( df["inserted"].values, df["o_rev_id"].values == int(revision))
            if(itr < (len(revisions) - 1) ):
                deleted_revision = revisions[itr + 1]
            else :
                deleted_revision = -1
            df["deleted"] = df["out"].apply(lambda x: deleted_revision in  set(x))
            store.put(revision, df, table=True)
        else:
            print(response.json())

/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '3317807'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['in', 'out', 'str']]

  if self.run_code(code, result):
/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '3317866'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  

/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '8688306'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '8859846'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '9325214'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will sti

/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '13727218'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '15127337'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/Users/aadilrasheed/anaconda/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: '15128604'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will 

In [237]:
with pd.HDFStore( filename, 'r') as store1:
    df_3317807 = store1.get("3317807")
    df_3317866 = store1.get("3317866")
    print(df_3317807.head(2))
    print("\n")   
    print(df_3317866.head(2))

   in  o_rev_id        out str  token_id  inserted  deleted
0  []   3317807  [3317921]  an         0      True    False
1  []   3317807  [3317921]  80         1      True    False


   in  o_rev_id        out str  token_id  inserted  deleted
0  []   3317807  [3317921]  an         0     False     True
1  []   3317807  [3317921]  80         1     False     True


In [142]:
inserted_array = np.array(df_3317866["inserted"])
inserted_pos = np.argwhere(inserted_array == True)
inserted_tokens = np.array(df_3317866 ["token_id"])[inserted_pos]

In [203]:
# inserted_pos = inserted_pos[0].reshape(1,108)
inserted_pos.shape
# inserted_tokens.shape

(53, 1)

In [198]:
inserted_token_pos = np.stack((inserted_tokens, inserted_pos), axis = -1)
inserted_token_pos = inserted_token_pos.reshape(inserted_token_pos.shape[0],2)
inserted_token_pos

array([[108,  52],
       [109,  53],
       [110,  54],
       [111,  55],
       [112,  56],
       [113,  57],
       [114, 109],
       [115, 110],
       [116, 111],
       [117, 112],
       [118, 113],
       [119, 114],
       [120, 115],
       [121, 116],
       [122, 117],
       [123, 118],
       [124, 119],
       [125, 120],
       [126, 121],
       [127, 122],
       [128, 123],
       [129, 124],
       [130, 125],
       [131, 126],
       [132, 127],
       [133, 128],
       [134, 129],
       [135, 130],
       [136, 131],
       [137, 132],
       [138, 133],
       [139, 134],
       [140, 135],
       [141, 136],
       [142, 137],
       [143, 138],
       [144, 139],
       [145, 140],
       [146, 141],
       [147, 142],
       [148, 143],
       [149, 144],
       [150, 145],
       [151, 146],
       [152, 147],
       [153, 148],
       [154, 149],
       [155, 150],
       [156, 151],
       [157, 152],
       [158, 153],
       [159, 154],
       [160,

### To cluster elements using position of there relative position in doc

In [155]:
from sklearn.cluster import DBSCAN

In [224]:
db = DBSCAN(eps=5, min_samples=1, p=1).fit(inserted_pos)
a = db.fit_predict(inserted_pos)
db.components_

array([[ 52],
       [ 53],
       [ 54],
       [ 55],
       [ 56],
       [ 57],
       [109],
       [110],
       [111],
       [112],
       [113],
       [114],
       [115],
       [116],
       [117],
       [118],
       [119],
       [120],
       [121],
       [122],
       [123],
       [124],
       [125],
       [126],
       [127],
       [128],
       [129],
       [130],
       [131],
       [132],
       [133],
       [134],
       [135],
       [136],
       [137],
       [138],
       [139],
       [140],
       [141],
       [142],
       [143],
       [144],
       [145],
       [146],
       [147],
       [148],
       [149],
       [150],
       [151],
       [152],
       [153],
       [154],
       [155]])

In [225]:
unique_clusters = set(db.labels_) - {-1}

{0, 1}

In [23]:
ls

Mama's_Family.h5                  wikiwho.ipynb
README.md                         wikiwho_context_clustering.ipynb
mama_700034288.h5                 wikiwho_neighbour.ipynb
test.h5


In [20]:
# rm Mama*